In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [9]:
df=pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
print(df.head())
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
df.info()

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [13]:
X=df.v2
Y=df.v1 #Encode the class Tables
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=Y.reshape(-1,1) #reshape array with 1 column and as many rows as necessary to accomadate the data
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object


In [20]:
import tensorflow as tf
#split test and training data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15)
#convert the train data into tokens
max_words=1000
max_len=150  #length of the sequence
tok=Tokenizer(num_words=max_words)
#updates internal vocabulary based on a list of texts
#in the case where texts contains lists,we assume each entry of the lists to be a token.
tok.fit_on_texts(X_train) #use tokenizer object to convert the text into tokens
#transforms each text in texts to the sequence of Integers
sequences=tok.texts_to_sequences(X_train)
print(sequences)


[[961, 385, 510, 6, 511, 161, 318, 621], [10, 3, 463, 11], [70, 80, 246, 177, 42, 41, 14, 62, 426, 1, 411, 35], [412, 1, 512, 226, 4, 15, 2, 202, 226, 63, 512, 73, 413], [56, 32, 590, 49], [29, 181, 158, 20], [51, 78, 741, 741, 9, 270, 23, 35, 132], [22, 38, 152, 5, 162, 1, 330, 39, 4, 43, 178, 15, 899, 79, 2, 835, 303, 50, 29, 114, 836, 697, 59, 47, 78, 477, 304, 837, 23, 1, 305, 31, 900, 18], [310, 319, 1, 57, 4, 7, 34, 622, 3, 33, 7], [39, 962, 2, 306, 4, 26, 159, 24, 698, 623, 40, 9, 265, 901, 71, 26, 44, 1, 271, 197], [35, 25, 787, 447], [107, 181, 181, 146, 63, 222, 214, 33, 16, 495, 203, 21, 83, 97, 464, 16], [38, 22, 29, 129, 115, 6, 86, 320, 1, 838, 345, 40, 183, 25, 1, 108, 1, 57, 39, 373, 124, 56, 38, 218, 18, 510], [321, 260, 788, 135, 478, 6, 260, 58, 374, 87, 699, 399, 541], [1, 322, 234, 167, 3, 479, 901, 3, 448, 53, 19, 465, 4, 139, 5, 963, 7, 26, 399, 174, 3, 571, 700, 98, 14, 964, 223, 5, 117, 34, 44, 13, 9, 56, 39, 323, 701, 153, 9, 4, 702, 2, 571, 25, 1, 346, 202, 1

In [21]:
sequences_matrix=tf.keras.utils.pad_sequences(sequences,maxlen=max_len)
print(sequences_matrix[0])
print(len(sequences_matrix),len(X_train))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 961 385
 510   6 511 161 318 621]
4736 4736


In [22]:
def RNN():
    inputs=Input(name='inputs',shape=[max_len])
    layer=Embedding(max_words,50,input_length=max_len)(inputs)
    layer=LSTM(64)(layer)
    layer=Dense(256,name='FC1')(layer)
    layer=Activation('relu')(layer)
    layer=Dropout(0.5)(layer)
    layer=Dense(1,name='out_layer')(layer)
    layer=Activation('sigmoid')(layer)
    model=Model(inputs=inputs,outputs=layer)
    return model

In [24]:
model=RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(),metrics=['accuracy'])
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)
          #callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [28]:
#prepare the test data by converting text to sequence
test_sequences=tok.texts_to_sequences(X_test)
test_sequences_matrix=tf.keras.utils.pad_sequences(test_sequences,maxlen=max_len)
accr=model.evaluate(test_sequences_matrix,Y_test)
print(accr)
print('Test Set \n Loss:{:0.3f}\n Accuracy: {:0.3f}'.format(accr[0],accr[1]))

27/27 [==============================] - 0s 7ms/step - loss: 0.0586 - accuracy: 0.9844
[0.05856948718428612, 0.9844497442245483]
Test Set 
 Loss:0.059
 Accuracy: 0.984
